In [22]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [23]:
date_cols = [
    'Auftragseingang', 
    'Auftragsende_SOLL',
    'AFO_Start_SOLL',
    'AFO_Ende_SOLL',
    'AFO_Start_IST',
    'AFO_Ende_IST'
]

data = pd.read_csv(
    '../data/processed/data_cleaned_maschinen_2.csv',
    parse_dates=date_cols,
    sep=',',
    low_memory=False
)

In [24]:
data.head()

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Maschinenkapazität,is_transport_ruesten,info
0,95a859f51cf541e0b4aed3a38bb93065,1,Steuerventilmodul,2013-09-20,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,False,NaN
1,ed4e40cb93c04d0f9bcb8f7ecdc8752a,1,Steuerventilmodul,2013-11-09,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,False,NaN
2,ce233ad078b9429b8bd40f09100e8ee0,1,Steuerventilmodul,2013-12-30,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,False,NaN
3,c6b0430e1b7b4f328f0ac195c3070390,1,Steuerventilmodul,2013-10-05,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,False,NaN
4,5a5b4b41d6d246cfbe862018b557702b,1,Steuerventilmodul,2013-07-27,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,False,NaN


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528238 entries, 0 to 1528237
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1528238 non-null  object        
 1   BauteilID                  1528238 non-null  int64         
 2   Bauteilbezeichnung         1528238 non-null  object        
 3   Auftragseingang            1528238 non-null  datetime64[ns]
 4   Priorität                  1528238 non-null  int64         
 5   Auftragsende_SOLL          1528238 non-null  datetime64[ns]
 6   Arbeitsschritt             1528238 non-null  int64         
 7   Arbeitsschrittbezeichnung  1528238 non-null  object        
 8   AFO_Start_SOLL             1528238 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1528238 non-null  datetime64[ns]
 10  AFO_Start_IST              1528238 non-null  datetime64[ns]
 11  AFO_Ende_IST               1528238 no

In [26]:
from pandas.tseries.offsets import BDay, CustomBusinessHour

# Arbeitszeit: Montag–Freitag, 07:00–15:00 Uhr
cbh = CustomBusinessHour(start='07:00', end='15:00', weekmask='Mon Tue Wed Thu Fri')

In [27]:
def arbeitszeit_diff_fast(start, end):
    if pd.isna(start) or pd.isna(end):
        return np.nan
    if end < start:
        return 0

    # Ganze Tage dazwischen (Mo–Fr)
    bdays = np.busday_count(start.date(), end.date(), weekmask='1111100')
    
    # Stundenanteil des Starttags
    start_hour = start.hour + start.minute/60
    end_hour = end.hour + end.minute/60

    start_hours = max(0, 15 - max(7, start_hour)) if start_hour < 15 else 0
    end_hours = max(0, min(15, end_hour) - 7) if end_hour > 7 else 0

    total_hours = bdays * 8 + end_hours - (8 - start_hours)
    return max(total_hours, 0)

In [28]:
data["AFO_Dauer_SOLL_Stunden"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Start_SOLL"], x["AFO_Ende_SOLL"]), axis=1
)

data["AFO_Dauer_IST_Stunden"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Start_IST"], x["AFO_Ende_IST"]), axis=1
)

data["AFO_Verspätung_Arbeitszeit"] = (
    data["AFO_Dauer_IST_Stunden"] - data["AFO_Dauer_SOLL_Stunden"]
)

data["Start_Delay_Arbeitszeit"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Start_SOLL"], x["AFO_Start_IST"]), axis=1
)

data["End_Delay_Arbeitszeit"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Ende_SOLL"], x["AFO_Ende_IST"]), axis=1
)
 

In [29]:
# 1️⃣ Auftragsebenen-Daten erzeugen: erster & letzter Arbeitsschritt + Start/Ende
auftrag_zeiten = (
    data.groupby("AuftragsID")
        .agg({
            "Arbeitsschritt": ["min", "max"],         # erster & letzter Schritt
            "AFO_Start_IST": "min",                  # erster tatsächlicher Start
            "AFO_Ende_IST": "max"                    # letzter tatsächlicher Abschluss
        })
        .reset_index()
)

# Spalten umbenennen für Klarheit
auftrag_zeiten.columns = [
    "AuftragsID",
    "AFO_Erster_Schritt",
    "AFO_Letzter_Schritt",
    "AFO_Start_IST_Erster",
    "AFO_Ende_IST_Letzter"
]

# 2️⃣ Gesamtlaufzeit in Kalendertagen (IST)
auftrag_zeiten["Auftrags_Laufzeit_IST_Tage"] = (
    (auftrag_zeiten["AFO_Ende_IST_Letzter"] - auftrag_zeiten["AFO_Start_IST_Erster"]).dt.total_seconds() / (24 * 3600)
)

# 3️⃣ Merge zurück in Hauptdatensatz
data = data.merge(
    auftrag_zeiten[[
        "AuftragsID",
        "AFO_Erster_Schritt",
        "AFO_Letzter_Schritt",
        "AFO_Start_IST_Erster",
        "AFO_Ende_IST_Letzter",
        "Auftrags_Laufzeit_IST_Tage"
    ]],
    on="AuftragsID",
    how="left"
)


In [30]:
# 4️⃣ Gesamtlaufzeit in Kalendertagen (SOLL)
auftrag_zeiten_soll = (
    data.groupby("AuftragsID")
        .agg({
            "AFO_Start_SOLL": "min",    # erster geplanter Start
            "AFO_Ende_SOLL": "max"      # letzter geplanter Abschluss
        })
        .reset_index()
)

# Berechnung der geplanten Laufzeit in Tagen
auftrag_zeiten_soll["Auftrags_Laufzeit_SOLL_Tage"] = (
    (auftrag_zeiten_soll["AFO_Ende_SOLL"] - auftrag_zeiten_soll["AFO_Start_SOLL"]).dt.total_seconds() / (24 * 3600)
)

# 5️⃣ Merge mit IST-Daten
auftrag_zeiten = auftrag_zeiten.merge(
    auftrag_zeiten_soll[["AuftragsID", "AFO_Start_SOLL", "AFO_Ende_SOLL", "Auftrags_Laufzeit_SOLL_Tage"]],
    on="AuftragsID",
    how="left"
)

# 6️⃣ Vergleich: IST vs. SOLL
auftrag_zeiten["Auftrags_Laufzeit_Abweichung_Tage"] = (
    auftrag_zeiten["Auftrags_Laufzeit_IST_Tage"] - auftrag_zeiten["Auftrags_Laufzeit_SOLL_Tage"]
)

# 7️⃣ Merge der erweiterten Daten zurück ins Haupt-DataFrame
data = data.merge(
    auftrag_zeiten[[
        "AuftragsID",
        "Auftrags_Laufzeit_SOLL_Tage",
        "Auftrags_Laufzeit_Abweichung_Tage"
    ]],
    on="AuftragsID",
    how="left"
)


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528238 entries, 0 to 1528237
Data columns (total 29 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   AuftragsID                         1528238 non-null  object        
 1   BauteilID                          1528238 non-null  int64         
 2   Bauteilbezeichnung                 1528238 non-null  object        
 3   Auftragseingang                    1528238 non-null  datetime64[ns]
 4   Priorität                          1528238 non-null  int64         
 5   Auftragsende_SOLL                  1528238 non-null  datetime64[ns]
 6   Arbeitsschritt                     1528238 non-null  int64         
 7   Arbeitsschrittbezeichnung          1528238 non-null  object        
 8   AFO_Start_SOLL                     1528238 non-null  datetime64[ns]
 9   AFO_Ende_SOLL                      1528238 non-null  datetime64[ns]
 10  AFO_St

In [32]:
auftrag_zeiten["Auftrags_Laufzeit_Ratio"] = (
    auftrag_zeiten["Auftrags_Laufzeit_IST_Tage"] / auftrag_zeiten["Auftrags_Laufzeit_SOLL_Tage"]
)

In [33]:
# Wie lange lag der Auftrag zwischen Auftragseingang und tatsächlichem Produktionsbeginn?

data["Wartezeit_vor_Beginn_Tage"] = (
    (data["AFO_Start_IST"] - data["Auftragseingang"]).dt.total_seconds() / (24 * 3600)
)


In [34]:
data["Pufferzeit_geplant_Tage"] = (
    (data["AFO_Start_SOLL"] - data["Auftragseingang"]).dt.total_seconds() / (24 * 3600)
)


In [35]:
data["AFO_Start_Wochentag_Num"] = data["AFO_Start_IST"].dt.weekday
data["AFO_Start_Stunde"] = data["AFO_Start_IST"].dt.hour
data["AFO_Kalenderwoche"] = data["AFO_Start_IST"].dt.isocalendar().week
data["AFO_Jahr"] = data["AFO_Start_IST"].dt.year
data["AFO_Ende_Stunde"] = data["AFO_Ende_IST"].dt.hour


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528238 entries, 0 to 1528237
Data columns (total 36 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   AuftragsID                         1528238 non-null  object        
 1   BauteilID                          1528238 non-null  int64         
 2   Bauteilbezeichnung                 1528238 non-null  object        
 3   Auftragseingang                    1528238 non-null  datetime64[ns]
 4   Priorität                          1528238 non-null  int64         
 5   Auftragsende_SOLL                  1528238 non-null  datetime64[ns]
 6   Arbeitsschritt                     1528238 non-null  int64         
 7   Arbeitsschrittbezeichnung          1528238 non-null  object        
 8   AFO_Start_SOLL                     1528238 non-null  datetime64[ns]
 9   AFO_Ende_SOLL                      1528238 non-null  datetime64[ns]
 10  AFO_St

In [37]:
def map_schicht(hour):
    if 7 <= hour < 11:
        return "Früh"
    elif 11 <= hour < 15:
        return "Mittag"
    else:
        return "Sonderschicht"

data["AFO_Schicht"] = data["AFO_Start_IST"].dt.hour.map(map_schicht)


In [38]:
data["Lieferabweichung_Stunden"] = (
    (data["AFO_Ende_IST"] - data["Auftragsende_SOLL"]).dt.total_seconds() / 3600
)


In [39]:
max_capacity = {
    "EWM": 18,
    "Fronius": 25,
    "Deckel Maho": 8,
    "DMG Mori": 8,
    "Lorch": 4
}

# Sicherstellen, dass Spalten vorhanden sind
if "Maschinenbezeichnung" in data.columns and "Maschinenkapazität" in data.columns:
    # Berechne relative Auslastung
    data["Maschinenkapazität_relativ"] = data.apply(
        lambda row: row["Maschinenkapazität"] / max_capacity.get(row["Maschinenbezeichnung"], np.nan),
        axis=1
    )
else:
    print("⚠️ Fehlende Spalten: Bitte sicherstellen, dass 'Maschinenbezeichnung' und 'Maschinenkapazität' vorhanden sind.")

In [40]:
data.drop(columns=["AuftragsID", "Bauteilbezeichnung", "Maschinenbezeichnung","Auftragsende_SOLL", "AFO_Ende_IST", "info", "Auftrags_Laufzeit_IST_Tage", "Auftrags_Laufzeit_SOLL_Tage", ], inplace=True)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528238 entries, 0 to 1528237
Data columns (total 31 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   BauteilID                          1528238 non-null  int64         
 1   Auftragseingang                    1528238 non-null  datetime64[ns]
 2   Priorität                          1528238 non-null  int64         
 3   Arbeitsschritt                     1528238 non-null  int64         
 4   Arbeitsschrittbezeichnung          1528238 non-null  object        
 5   AFO_Start_SOLL                     1528238 non-null  datetime64[ns]
 6   AFO_Ende_SOLL                      1528238 non-null  datetime64[ns]
 7   AFO_Start_IST                      1528238 non-null  datetime64[ns]
 8   MaschinenID                        538654 non-null   float64       
 9   Maschinenkapazität                 538654 non-null   float64       
 10  is_tra

In [42]:
data.to_csv('../data/processed/data_feature_zeit_3.csv', index=False)